<a href="https://colab.research.google.com/github/Theanh130124/DataMining_Colab/blob/main/2251012005_Tr%E1%BA%A7n_Th%E1%BA%BF_Anh_BaiCaNhan_Tuan6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Đọc dữ liệu từ tệp gốc
file_path = '/content/Weather HCMC2020To8-2024 Full.csv'
weather_data = pd.read_csv(file_path, encoding='latin1')

# Loại bỏ các cột không cần thiết
columns_to_drop = ['Unnamed: 21', 'MoTa', 'N?ng l??ng m?t tr?i', 'MatTrang', 'BinhMinh', 'HoangHon']
weather_data = weather_data.drop(columns=columns_to_drop)

# Đổi tên cột để dễ làm việc hơn (nếu cần)
weather_data.columns = weather_data.columns.str.strip()  # Loại bỏ khoảng trắng đầu/cuối cột

# Chuyển cột 'Ngày' về định dạng datetime
weather_data['Ngày'] = pd.to_datetime(weather_data['Ngày'], format='%d-%m-%y', errors='coerce')

# Kiểm tra và chuyển đổi các cột số từ chuỗi sang float
numeric_columns = ['NhietDoMax', 'NhietDoMin', 'NhietDoTB', 'NhietDoCamNhanMax',
                   'NhietDoCamNhanMin', 'NhietoCamNhan', 'SuongMu', 'DoAm', 'GioGiat',
                   'VanTocGio', 'MucNuocBien', 'MayChePhu', 'TamNhin', 'BucXa', 'ChiSoUV']

for col in numeric_columns:
    weather_data[col] = pd.to_numeric(weather_data[col], errors='coerce')

# Tách dữ liệu thành Train (2020-2023) và Test (2024)
train_data = weather_data[weather_data['Ngày'].dt.year.isin([2020, 2021, 2022, 2023])]
test_data = weather_data[weather_data['Ngày'].dt.year == 2024]
# Tạo thư mục /mnt/data nếu nó chưa tồn tại
!mkdir -p /mnt/data

# Xuất dữ liệu ra hai tệp CSV
train_path = '/mnt/data/Train_Weather_HCMC.csv'
test_path = '/mnt/data/Test_Weather_HCMC.csv'
train_data.to_csv(train_path, index=False, encoding='utf-8')
test_data.to_csv(test_path, index=False, encoding='utf-8')

print(f"Train file saved to: {train_path}")
print(f"Test file saved to: {test_path}")


Train file saved to: /mnt/data/Train_Weather_HCMC.csv
Test file saved to: /mnt/data/Test_Weather_HCMC.csv


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

# Đường dẫn tệp
train_path = '/mnt/data/Train_Weather_HCMC.csv'
test_path = '/mnt/data/Test_Weather_HCMC.csv'

# Đọc dữ liệu Train và Test
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Hàm tiền xử lý dữ liệu
def preprocess_data(data):
    # Loại bỏ các hàng thiếu giá trị mục tiêu
    data = data.dropna(subset=['Mua/KhngMua'])

    # Mã hóa nhãn mục tiêu: "rain" -> 1 (Mưa), các giá trị khác -> 0 (Không mưa)
    data['Mua/KhngMua'] = data['Mua/KhngMua'].apply(lambda x: 1 if x == 'rain' else 0)

    # Loại bỏ các cột không cần thiết
    drop_columns = ['Ngày', 'HuongGio']  # Cột 'Ngày' và 'HuongGio' không sử dụng
    data = data.drop(columns=drop_columns, errors='ignore')

    # Điền giá trị thiếu trong các cột số bằng giá trị trung bình
    for col in data.columns:
        if data[col].dtype in ['float64', 'int64']:
            data[col].fillna(data[col].mean(), inplace=True)

    return data

# Tiền xử lý dữ liệu Train và Test
train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

# Kiểm tra phân bố nhãn trước huấn luyện
print("Phân bố nhãn trong tập Train:")
print(train_data['Mua/KhngMua'].value_counts())
print("\nPhân bố nhãn trong tập Test:")
print(test_data['Mua/KhngMua'].value_counts())

# Phân chia đầu vào (X) và đầu ra (y)
X_train = train_data.drop(columns=['Mua/KhngMua'])
y_train = train_data['Mua/KhngMua']

X_test = test_data.drop(columns=['Mua/KhngMua'])
y_test = test_data['Mua/KhngMua']

# 1. Tối ưu hóa siêu tham số bằng GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None],
    'max_features': ['sqrt', 'log2', None],
    'class_weight': ['balanced', 'balanced_subsample']
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='f1',  # Ưu tiên tối ưu F1-score
    cv=5,  # Chia tập train thành 5 tập con để đánh giá
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# 2. Chọn đặc trưng quan trọng
importances = best_model.feature_importances_
features = X_train.columns
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)

selected_features = feature_importances[feature_importances['Importance'] > 0.1]['Feature']
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# 3. Cân bằng dữ liệu với SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_selected, y_train)

# Huấn luyện lại mô hình với dữ liệu cân bằng
best_model.fit(X_train_balanced, y_train_balanced)

# Dự đoán trên tập Test
y_pred = best_model.predict(X_test_selected)

# 4. Đánh giá mô hình
print("\nAccuracy after improvements:", accuracy_score(y_test, y_pred))
print("\nClassification Report after improvements:\n", classification_report(y_test, y_pred))

# Hiển thị ma trận nhầm lẫn
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Không Mưa', 'Mưa'], yticklabels=['Không Mưa', 'Mưa'])
plt.title("Confusion Matrix")
plt.xlabel("Dự đoán")
plt.ylabel("Thực tế")
plt.show()

# 5. Đánh giá AUC
y_pred_proba = best_model.predict_proba(X_test_selected)[:, 1]
auc_score = roc_auc_score(y_test, y_pred_proba)
print("\nROC-AUC Score:", auc_score)

# Vẽ đường ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}", color="blue")
plt.plot([0, 1], [0, 1], linestyle='--', color='red')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()

# Hiển thị kết quả dự đoán và nhãn thực tế
results = pd.DataFrame({'Nhãn thực tế': y_test, 'Dự đoán': y_pred})
results['Dự đoán'] = results['Dự đoán'].apply(lambda x: 'Mưa' if x == 1 else 'Không Mưa')
results['Nhãn thực tế'] = results['Nhãn thực tế'].apply(lambda x: 'Mưa' if x == 1 else 'Không Mưa')
print("\nDự đoán và Nhãn thực tế:")
print(results)


Phân bố nhãn trong tập Train:
Mua/KhngMua
1    1104
Name: count, dtype: int64

Phân bố nhãn trong tập Test:
Mua/KhngMua
1    98
Name: count, dtype: int64
Fitting 5 folds for each of 54 candidates, totalling 270 fits


<ipython-input-33-6a5dcc7cc56c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Mua/KhngMua'] = data['Mua/KhngMua'].apply(lambda x: 1 if x == 'rain' else 0)
<ipython-input-33-6a5dcc7cc56c>:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplac

Best Parameters: {'class_weight': 'balanced', 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 50}

Feature Importances:
              Feature  Importance
0          NhietDoMax         0.0
1          NhietDoMin         0.0
2           NhietDoTB         0.0
3   NhietDoCamNhanMax         0.0
4   NhietDoCamNhanMin         0.0
5       NhietoCamNhan         0.0
6             SuongMu         0.0
7                DoAm         0.0
8             GioGiat         0.0
9           VanTocGio         0.0
10        MucNuocBien         0.0
11          MayChePhu         0.0
12            TamNhin         0.0
13              BucXa         0.0
14            ChiSoUV         0.0


ValueError: at least one array or dtype is required

IndexError: index 1 is out of bounds for axis 1 with size 1